In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib as matplot
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

from keras.models import Model, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support,accuracy_score)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, validation_curve
from sklearn.preprocessing import LabelEncoder,normalize,StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from logitboost import LogitBoost
from IPython.display import display, Image, SVG, Math, YouTubeVideo
from data_exploration import explore
from feature_engineering import transformation

In [22]:
test = pd.read_csv('UNSW_NB15_training-set.csv')
train = pd.read_csv('UNSW_NB15_testing-set.csv')

In [23]:
combined_data_1 = pd.concat([train, test]).drop(['id'],axis=1)

In [24]:
train = train.where(train['service'] == "http").dropna()
test = test.where(test['service'] == "http").dropna()

In [25]:
combined_data = pd.concat([train, test]).drop(['id'],axis=1)

In [6]:
combined_data.shape

(27011, 44)

In [26]:
combined_data.head(3)

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
35,0.983874,tcp,http,FIN,10.0,8.0,816.0,1172.0,17.278635,62.0,...,1.0,2.0,0.0,0.0,1.0,1.0,3.0,0.0,Normal,0.0
40,1.535254,tcp,http,FIN,10.0,10.0,826.0,1266.0,12.375802,62.0,...,1.0,2.0,0.0,0.0,1.0,1.0,3.0,0.0,Normal,0.0
45,1.059359,tcp,http,FIN,10.0,8.0,830.0,1134.0,16.047441,62.0,...,1.0,1.0,0.0,0.0,1.0,3.0,1.0,0.0,Normal,0.0


In [9]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27011 entries, 35 to 175214
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                27011 non-null  float64
 1   proto              27011 non-null  object 
 2   service            27011 non-null  object 
 3   state              27011 non-null  object 
 4   spkts              27011 non-null  float64
 5   dpkts              27011 non-null  float64
 6   sbytes             27011 non-null  float64
 7   dbytes             27011 non-null  float64
 8   rate               27011 non-null  float64
 9   sttl               27011 non-null  float64
 10  dttl               27011 non-null  float64
 11  sload              27011 non-null  float64
 12  dload              27011 non-null  float64
 13  sloss              27011 non-null  float64
 14  dloss              27011 non-null  float64
 15  sinpkt             27011 non-null  float64
 16  dinpkt             2

In [6]:
combined_data.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,...,1,1,0,0,0,1,1,0,Normal,0
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,...,1,2,0,0,0,1,6,0,Normal,0
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,...,1,3,0,0,0,2,6,0,Normal,0
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,3,1,1,0,2,1,0,Normal,0
4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,...,1,40,0,0,0,2,39,0,Normal,0


In [27]:
tmp = train.where(train['attack_cat'] == "Normal").dropna()
len(tmp['attack_cat'])/len(train['attack_cat'])

0.48425244358634006

In [28]:
tmp = test.where(test['attack_cat'] == "Normal").dropna()
len(tmp['attack_cat'])/len(test['attack_cat'])

0.2856227301858577

In [44]:
le1 = LabelEncoder()
le = LabelEncoder()

#vector = combined_data['attack_cat']

#print("attack cat:", set(list(vector))) # use print to make it print on single line 

#combined_data['attack_cat'] = le1.fit_transform(vector)
combined_data['proto'] = le.fit_transform(combined_data['proto'])
combined_data['service'] = le.fit_transform(combined_data['service'])
combined_data['state'] = le.fit_transform(combined_data['state'])

In [45]:
attack = ['attack_cat']

In [119]:
fd = pd.get_dummies(combined_data_1,columns=attack,drop_first=False)

In [46]:
fd = pd.get_dummies(combined_data,columns=attack,drop_first=False)

In [47]:
fd.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,label,attack_cat_Analysis,attack_cat_Backdoor,attack_cat_DoS,attack_cat_Exploits,attack_cat_Fuzzers,attack_cat_Generic,attack_cat_Normal,attack_cat_Reconnaissance,attack_cat_Worms
35,0.983874,0,0,1,10.0,8.0,816.0,1172.0,17.278635,62.0,...,0.0,0,0,0,0,0,0,1,0,0
40,1.535254,0,0,1,10.0,10.0,826.0,1266.0,12.375802,62.0,...,0.0,0,0,0,0,0,0,1,0,0
45,1.059359,0,0,1,10.0,8.0,830.0,1134.0,16.047441,62.0,...,0.0,0,0,0,0,0,0,1,0,0
49,0.990548,0,0,1,10.0,10.0,804.0,1414.0,19.181301,62.0,...,0.0,0,0,0,0,0,0,1,0,0
72,1.303518,0,0,1,12.0,8.0,898.0,1120.0,14.575939,62.0,...,0.0,0,0,0,0,0,0,1,0,0


In [48]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27011 entries, 35 to 175214
Data columns (total 52 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dur                        27011 non-null  float64
 1   proto                      27011 non-null  int32  
 2   service                    27011 non-null  int32  
 3   state                      27011 non-null  int32  
 4   spkts                      27011 non-null  float64
 5   dpkts                      27011 non-null  float64
 6   sbytes                     27011 non-null  float64
 7   dbytes                     27011 non-null  float64
 8   rate                       27011 non-null  float64
 9   sttl                       27011 non-null  float64
 10  dttl                       27011 non-null  float64
 11  sload                      27011 non-null  float64
 12  dload                      27011 non-null  float64
 13  sloss                      27011 non-null  f

In [99]:
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True  positive  = ', cm[0][0])
    print('False positive  = ', cm[0][1])
    print('False negative  = ', cm[1][0])
    print('True  negative  = ', cm[1][1])

In [97]:
def detection_rate(y_true, y_pred):
    CM = metrics.confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    return TP/(TP+FN)

def false_positive_rate(y_true, y_pred):
    CM = metrics.confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    return FP/(FP+TN)



def results(y_test, y_pred):
    print(f"Accuracy:--> {metrics.accuracy_score(y_test, y_pred)*100}")
    print(f"F1-score:--> {metrics.f1_score(y_test, y_pred)*100}")
    # print(metrics.classification_report(y_test, y_pred))
    print("DR:--------> {0}".format(detection_rate(y_test, y_pred)*100))
    print("FAR:------->  {0}".format(false_positive_rate(y_test, y_pred)*100))

# Attack-->Analysis

In [49]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_Analysis']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [50]:
data_x = combined_data_reduced.drop(['attack_cat_Analysis'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_Analysis']]

In [51]:
data_x.head()

,proto,service,dpkts,sttl,smean,dmean,trans_depth,response_body_len,ct_state_ttl,ct_flw_http_mthd
35,0,0,8.0,62.0,82.0,147.0,1.0,184.0,1.0,1.0
40,0,0,10.0,62.0,83.0,127.0,1.0,187.0,1.0,1.0
45,0,0,8.0,62.0,83.0,142.0,1.0,165.0,1.0,1.0
49,0,0,10.0,62.0,80.0,141.0,1.0,261.0,1.0,1.0
72,0,0,8.0,62.0,75.0,140.0,1.0,157.0,1.0,1.0


In [52]:
data_y.tail()

,attack_cat_Analysis
174998,0
175039,0
175076,0
175138,0
175214,0


In [53]:
## Get the Fraud and the normal dataset 

fraud = combined_data_reduced[combined_data_reduced['attack_cat_Analysis']==1]

normal = combined_data_reduced[combined_data_reduced['attack_cat_Analysis']==0]

In [54]:
print(fraud.shape,normal.shape)

(558, 11) (26453, 11)


In [55]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.55, random_state=42) # TODO

In [56]:
Train_1 = y_train[y_train['attack_cat_Analysis']==1]

Test_1 = y_test[y_test['attack_cat_Analysis']==1]

In [57]:
print(Train_1.shape,Test_1.shape)

(237, 1) (321, 1)


In [58]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [59]:
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(12154, 10)

(12154, 1)

(14857, 10)

(14857, 1)

In [60]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [61]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [62]:
analysis_rfc_pred = rfc.predict(X_test)

In [63]:
print("Accuracy:",metrics.accuracy_score(y_test, analysis_rfc_pred)*100,"%")

Accuracy: 97.99421148280271 %


In [64]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 98.26 %
Standard Deviation: 0.15 %


In [100]:
print_confusion_matrix(y_test, analysis_rfc_pred)

True  positive  =  14447
False positive  =  89
False negative  =  209
True  negative  =  112


In [101]:
results(y_test, analysis_rfc_pred)

Accuracy:--> 97.99421148280271
F1-score:--> 42.91187739463602
DR:--------> 34.890965732087224
FAR:------->  0.612272977435333


# Backdoor

In [102]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_Backdoor']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [103]:
data_x = combined_data_reduced.drop(['attack_cat_Backdoor'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_Backdoor']]

In [113]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.55, random_state=42) # TODO

In [114]:
Train_1 = y_train[y_train['attack_cat_Backdoor']==1]

Test_1 = y_test[y_test['attack_cat_Backdoor']==1]

In [115]:
print(Train_1.shape,Test_1.shape)

(47, 1) (45, 1)


In [116]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [117]:
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(12154, 10)

(12154, 1)

(14857, 10)

(14857, 1)

In [118]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [119]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [120]:
rfc_pred = rfc.predict(X_test)

In [121]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred)*100,"%")

Accuracy: 99.60961163088106 %


In [122]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 99.69 %
Standard Deviation: 0.11 %


In [123]:
print_confusion_matrix(y_test, rfc_pred)

True  positive  =  14785
False positive  =  27
False negative  =  31
True  negative  =  14


In [124]:
results(y_test, rfc_pred)

Accuracy:--> 99.60961163088106
F1-score:--> 32.558139534883715
DR:--------> 31.11111111111111
FAR:------->  0.1822846340804753


# Fuzzers

In [125]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_Fuzzers']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [126]:
data_x = combined_data_reduced.drop(['attack_cat_Fuzzers'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_Fuzzers']]

In [134]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.55, random_state=42) # TODO

In [135]:
Train_1 = y_train[y_train['attack_cat_Fuzzers']==1]

Test_1 = y_test[y_test['attack_cat_Fuzzers']==1]

In [136]:
print(Train_1.shape,Test_1.shape)

(514, 1) (573, 1)


In [137]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [138]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [139]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [140]:
rfc_pred = rfc.predict(X_test)

In [141]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred)*100,"%")

Accuracy: 97.67786228713737 %


In [142]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 97.63 %
Standard Deviation: 0.43 %


In [143]:
print_confusion_matrix(y_test, rfc_pred)

True  positive  =  14034
False positive  =  250
False negative  =  95
True  negative  =  478


In [144]:
results(y_test, rfc_pred)

Accuracy:--> 97.67786228713737
F1-score:--> 73.48193697156034
DR:--------> 83.42059336823735
FAR:------->  1.7502100252030244


# Reconnaissance

In [145]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_Reconnaissance']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [146]:
data_x = combined_data_reduced.drop(['attack_cat_Reconnaissance'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_Reconnaissance']]

In [155]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.50, random_state=42) # TOD

In [156]:
Train_1 = y_train[y_train['attack_cat_Reconnaissance']==1]

Test_1 = y_test[y_test['attack_cat_Reconnaissance']==1]

In [157]:
print(Train_1.shape,Test_1.shape)

(1032, 1) (1041, 1)


In [158]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [159]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [160]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [161]:
rfc_pred = rfc.predict(X_test)

In [162]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred)*100,"%")

Accuracy: 93.74352139789724 %


In [163]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 93.62 %
Standard Deviation: 0.31 %


In [164]:
print_confusion_matrix(y_test, rfc_pred)

True  positive  =  12129
False positive  =  336
False negative  =  509
True  negative  =  532


In [165]:
results(y_test, rfc_pred)

Accuracy:--> 93.74352139789724
F1-score:--> 55.735987427972766
DR:--------> 51.10470701248799
FAR:------->  2.6955475330926593


# Exploits

In [166]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_Exploits']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [167]:
data_x = combined_data_reduced.drop(['attack_cat_Exploits'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_Exploits']]

In [177]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.51, random_state=42) # TODO

In [178]:
Train_1 = y_train[y_train['attack_cat_Exploits']==1]

Test_1 = y_test[y_test['attack_cat_Exploits']==1]

In [179]:
print(Train_1.shape,Test_1.shape)

(5698, 1) (5783, 1)


In [180]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [181]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [182]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [183]:
rfc_pred = rfc.predict(X_test)

In [184]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred)*100,"%")

Accuracy: 84.38588850174216 %


In [185]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 84.35 %
Standard Deviation: 0.85 %


In [186]:
print_confusion_matrix(y_test, rfc_pred)

True  positive  =  6871
False positive  =  1122
False negative  =  1029
True  negative  =  4754


In [187]:
results(y_test, rfc_pred)

Accuracy:--> 84.38588850174216
F1-score:--> 81.55073333905139
DR:--------> 82.20646723154073
FAR:------->  14.037282622294509


# DoS

In [188]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_DoS']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [189]:
data_x = combined_data_reduced.drop(['attack_cat_DoS'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_DoS']]

In [193]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.51, random_state=42) # TODO

In [194]:
Train_1 = y_train[y_train['attack_cat_DoS']==1]

Test_1 = y_test[y_test['attack_cat_DoS']==1]

In [195]:
print(Train_1.shape,Test_1.shape)

(847, 1) (862, 1)


In [196]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [197]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [198]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [199]:
rfc_pred = rfc.predict(X_test)

In [200]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred)*100,"%")

Accuracy: 94.23635307781649 %


In [201]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 94.07 %
Standard Deviation: 0.38 %


In [202]:
print_confusion_matrix(y_test, rfc_pred)

True  positive  =  12676
False positive  =  238
False negative  =  556
True  negative  =  306


In [203]:
results(y_test, rfc_pred)

Accuracy:--> 94.23635307781649
F1-score:--> 43.52773826458037
DR:--------> 35.498839907192576
FAR:------->  1.842961127458572


# Worms

In [204]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_Worms']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [205]:
data_x = combined_data_reduced.drop(['attack_cat_Worms'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_Worms']]

In [210]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.51, random_state=42) # TODO

In [211]:
Train_1 = y_train[y_train['attack_cat_Worms']==1]

Test_1 = y_test[y_test['attack_cat_Worms']==1]

In [212]:
print(Train_1.shape,Test_1.shape)

(75, 1) (73, 1)


In [213]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [214]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [215]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [216]:
rfc_pred = rfc.predict(X_test)

In [217]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred)*100,"%")

Accuracy: 99.59349593495935 %


In [218]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 99.64 %
Standard Deviation: 0.08 %


In [219]:
print_confusion_matrix(y_test, rfc_pred)

True  positive  =  13677
False positive  =  26
False negative  =  30
True  negative  =  43


In [220]:
results(y_test, rfc_pred)

Accuracy:--> 99.59349593495935
F1-score:--> 60.56338028169014
DR:--------> 58.9041095890411
FAR:------->  0.18973947310807854


# Generic

In [221]:
combined_data_reduced=fd.loc[:,['proto', 'service', 'dpkts', 'sttl', 'smean', 'dmean', 'trans_depth',
                                'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd','attack_cat_Generic']]
fd.shape
combined_data_reduced.shape

(27011, 52)

(27011, 11)

In [222]:
data_x = combined_data_reduced.drop(['attack_cat_Generic'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['attack_cat_Generic']]

In [223]:
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.50, random_state=42) # TODO

In [224]:
Train_1 = y_train[y_train['attack_cat_Generic']==1]

Test_1 = y_test[y_test['attack_cat_Generic']==1]

In [225]:
print(Train_1.shape,Test_1.shape)

(245, 1) (257, 1)


In [226]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [227]:
rfc = RandomForestClassifier(n_estimators=25, random_state=1)

In [228]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=25, random_state=1)

In [229]:
rfc_pred = rfc.predict(X_test)

In [230]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred)*100,"%")

Accuracy: 98.77832074633496 %


In [231]:
accuracies = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 98.70 %
Standard Deviation: 0.16 %


In [232]:
print_confusion_matrix(y_test, rfc_pred)

True  positive  =  13209
False positive  =  40
False negative  =  125
True  negative  =  132


In [233]:
results(y_test, rfc_pred)

Accuracy:--> 98.77832074633496
F1-score:--> 61.53846153846154
DR:--------> 51.36186770428015
FAR:------->  0.30190957808136465
